In [509]:
import nltk,re,time,umap,collections,os,string,sys,mlflow
sys.path.append(os.path.abspath(".."))
from modules import utils,modeling

from sentence_transformers import SentenceTransformer,util
import plotly.express as px
import pandas as pd
import glob

from sklearn.metrics import completeness_score,homogeneity_score, v_measure_score,normalized_mutual_info_score,adjusted_mutual_info_score,silhouette_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from summa.summarizer import summarize
from summa.keywords import keywords
from hdbscan import HDBSCAN
from sentence_transformers import models
from sklearn.preprocessing import normalize
from torch import nn
import mlflow.sklearn
from  mlflow.tracking import MlflowClient
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from textwrap import wrap
stopwords = utils.load_stopwords()

remote_server="http://ec2-18-193-70-141.eu-central-1.compute.amazonaws.com"
client = MlflowClient(remote_server)
mlflow.set_tracking_uri(remote_server)


%matplotlib inline
%load_ext autoreload
%autoreload 2

%reload_ext utils
%reload_ext modeling
os.environ["MLFLOW_S3_ENDPOINT_URL"]="http://s3.eu-central-1.amazonaws.com"
os.environ["MLFLOW_TRACKING_USERNAME"]="mlflow"
os.environ["MLFLOW_TRACKING_PASSWORD"]="mlflow"

Number of stopwords 607
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [510]:
df= utils.load_text_data("../data/20k_31-10.csv").head(2000).sort_values("created_at",ascending=False).drop_duplicates("headline").dropna()
#df.created_at=df.created_at.dt.date.apply(str)

df.head(10)

,kicker,text,headline,seo_title,created_at,kicker_headline_NER,seo_title_NER,text_NER
article_uid,,,,,,,,
77f34d307c5d8fd10148ab14ff1647e812770638ada4fcf6857bffcbf4bb4012,Tourismus-Beauftragter fordert,Weihnachten­ sollen Hotels­ und Restaurants un...,Weihnachten sollen Restaurants und Hotels geöf...,Tourismus-Beauftragter fordert - Weihnachten s...,2020-10-31 23:11:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
662b9c5e9a24f491fcdc959f15e1e62018fb47e31209f207ad199ec146320c02,„Das Supertalent“,Was war das denn? Beim „Supertalent“ sorgte am...,Dieser Auftritt war zum Fürchten,„Das Supertalent“: Dieser Auftritt war zum Für...,2020-10-31 23:11:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
a26bac4d411fc5092af65c97024a8f590dc1caf5909b11122ed2e53df1b3d220,Erster Ministerpräsident warnt,Fast jeden Tag verzeichnet Deutschland neue Re...,„Kliniken vor dem Kollaps!“,Erster Ministerpräsident warnt - „Kliniken vor...,2020-10-31 23:11:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
0e6b74e1a268960d852e799294c4d707b51d5da424f4b4b61bdfd00943c328e1,Standpunkt,Ab morgen wird das gesellschaftliche Leben in ...,Was kommt nach dem Lockdown?,Standpunkt - Was kommt nach dem Lockdown? -Po...,2020-10-31 23:11:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
d304b07fab9d151087ab3a7f08b023f08f0f00eb7ad3c863e2b06dcf930d920d,Er sitzt in Frankreich fest,"Wenn seine Familie ihn sehen will, muss sie am...",Ausgangssperre für „Tatort“-Star,„Tatort“-Star Barnaby Metschurat sitzt in Fran...,2020-10-31 23:11:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
e9874ad843ec17b3c3a1d6f00204a06fa45b79fb644d10ce351ceb993d53e466,Mutter startet Kampagne gegen Alkohol am Steuer,Es waren nur drei Worte. Der Anruf am frühen M...,"Wenn Janines Tod ein Leben rettet, habe ich es...",Mutter startet Kampagne gegen Alkohol am Steue...,2020-10-31 22:45:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
660123306c159396af477697e907918b2ac0e4b176c8865520d4381cce190d32,Nach Erdbeben in der Türkei suchen Retter nach...,Mitten in den Trümmern von Izmir brandet plötz...,"„Wenn du meine Stimme hörst, klopfe dreimal“",Erdbeben in der Türkei: „Wenn du meine Stimme ...,2020-10-31 22:40:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
8bc6c27f485fa096211847d3d8c146440aebdbbc590cd02be3d21bdecdc413c4,Reifen-Flüsterer und Wackel-Auto bei Haas,Wo landet Mick Schumacher (21) in der nächsten...,Was Mick in der Formel 1 erwarten würde,Formel 1: Was Mick Schumacher bei Haas erwarte...,2020-10-31 22:29:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
ea3f2b6c9b1c28e6f39bda7f13f794027a2125b8f6b5b29e2e9018bac8700d1c,Hertha hofft auf Cordoba,Hertha BSC hat mit nur drei Punkten nach fünf ...,Sieg gegen Wolfsburg – oder Labbadia kriegt di...,Hertha BSC: Sieg gegen Wolfsburg – oder Labbad...,2020-10-31 22:29:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""me

# Topic Thunder Presentation Notebook

Here we are going to explore the basic functionality of the topic extracion and clustering on BILD articles. 

### Let's load last 2000 articles published on the Bild.de website on before 31/10/2020

In [456]:
sum_pdf= list(map(lambda x: summarize(x,language="german",ratio=0.9,additional_stopwords=stopwords).replace("\n",''),pdf))


In [402]:

df['_text'] = df['seo_title'].apply(lambda x: utils.remove_seo_title_marker(x,True)) +". "+ df["text"]

pdf = utils.preprocess_articles_for_bert(df,"_text",lower=False)
_pdf = pd.DataFrame(pdf,index=df.index)
for x in range(7):
    print("{}. {}...".format(x,pdf[x][:50]))

0. Tourismus Beauftragter fordert Weihnachten sollen ...
1. Das Supertalent Dieser Auftritt war zum Fürchten. ...
2. Erster Ministerpräsident warnt Kliniken vor dem Ko...
3. Standpunkt Was kommt nach dem Lockdown?. Ab morgen...
4. Tatort Star Barnaby Metschurat sitzt in Frankreich...
5. Mutter startet Kampagne gegen Alkohol am Steuer We...
6. Erdbeben in der Türkei Wenn du meine Stimme hörst,...


In [565]:
model = models.Transformer( 'T-Systems-onsite/bert-german-dbmdz-uncased-sentence-stsb',max_seq_length=225)
embeddings = modeling.get_sentence_embeddings(pdf,model,False)


--- Embedding dimension 768
--- 1976 Documnets encoded 495.3645420074463 seconds ---


In [566]:
model.max_seq_length =250

In [567]:
print(embeddings.shape)
_pdf["embedding"] = pd.Series(embeddings.tolist(),index=_pdf.index)

results,cluster_labels = modeling.cluster_and_reduce(embeddings,n_components_clustering=384,min_cluster_size=3,n_neighbors=8, min_samples= 4,alpha=0.75)

(1976, 768)


In [568]:
def summarize_NER(docs_per_topic,docs_df):
    ent = docs_per_topic.text_ne.values + docs_per_topic.kicker_headline_ne.values *2 + docs_per_topic.seo_title_ne.values * 3
    tf_idf, count = utils.c_tf_idf(ent, m=len(pdf),ngram_range=(1,2),remove_stop_words=True)
    top_n_words = utils.extract_top_n_words_per_topic(tf_idf, count, docs_per_topic.reset_index(), n=20)
    top_6_words_per_topic = pd.DataFrame([(k," | ".join([e[0] for e in v][:10]))  for (k,v) in top_n_words.items()],columns=["Topic","top_n_words"])
    top_6_words_per_topic["Topic"]=top_6_words_per_topic["Topic"].apply(int)
    topic_sizes = utils.extract_topic_sizes(docs_df)
    topic_sizes["Topic"]=topic_sizes["Topic"].apply(int)
#    top_6_words_per_topic=top_6_words_per_topic.set_index("Topic")
    #print(top_6_words_per_topic)
    
    return pd.merge(top_6_words_per_topic,topic_sizes,left_on="Topic",right_on="Topic")#.join(topic_sizes,how="left",on = "Topic",rsuffix="_r")

In [574]:



def sumarize_pagerank(docs_per_topic):
    res=[]
    ctr =0
    for tpc in docs_per_topic.index.to_list():
        if int(tpc) != -1:
            abstract = keywords(docs_per_topic.loc[tpc].raw_text,language="german",ratio=0.01,additional_stopwords=stopwords)
        else:
            abstract=" NONE "
        print(">>> Summarized {}/{} topic - {}".format(ctr,len(docs_per_topic.index.to_list()),abstract),end='\r')
        res.append([tpc,abstract])
        ctr += 1
        df = pd.DataFrame(res,columns=["Topic","top_n_words"])
        
    return df

@interact_manual(min_cluster_size=(1,10,1), min_samples=(1,12,1),cluster_selection_epsilon=(0,0.990,0.01),alpha=1.0,umap_model=[x for x in  glob.glob("../models/*/umap_*k*.pkl")])
def g(min_cluster_size=3, min_samples=2,cluster_selection_epsilon=0.14,alpha=1,seaborn= False,umap_model="bert-german-dbmdz-uncased-sentence-stsb/umap_100k_6-neighbors_256-comps.pkl"):
    umap_data,labels = modeling.load_umap_and_cluster(embeddings,umap_model=umap_model, min_samples=min_samples,
                                                      min_cluster_size=min_cluster_size,
                                                      cluster_selection_epsilon=cluster_selection_epsilon,
                                                      cluster_selection_method="leaf", alpha=alpha,
                                                     allow_single_cluster=False)
    
    docs_df = utils.relink_data_after_clustering(umap_data,df,labels)
    docs_df=docs_df.rename(columns={"topic_number":"Topic"})
    docs_df["Topic"] = docs_df["Topic"].astype(int)
    docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'raw_text': ' '.join,
                                                                       "created_at":"count",
                                                                   "seo_title_ne":lambda x: ", ".join([ent["text"] for ent in (utils.flatten([utils.parse_google_named_entities(el) for el in x ]))]),
                                                                    "kicker_headline_ne":lambda x: ", ".join([ent["text"] for ent in (utils.flatten([utils.parse_google_named_entities(el) for el in x ]))]),
                                                                    "text_ne": lambda x: ", ".join([ent["text"] for ent in (utils.flatten([utils.parse_google_named_entities(el) for el in x ]))])
                                                                  }).set_index("Topic")
    
    cluster_labels_google = summarize_NER(docs_per_topic,docs_df) # Cluster desc 
    #cluster_labels_google = sumarize_pagerank(docs_per_topic) # Cluster desc 

    #=res = pd.merge(docs_df,docs_per_topic.reset_index()[["Topic",'created_at','seo_title_ne']],left_on="Topic",suffixes=(None,"_count"),right_on = "Topic")
    #res=res.join(cluster_labels_google,how='left',on="Topic",rsuffix="_r")
    res = pd.merge(docs_df,cluster_labels_google,left_on="Topic",right_on="Topic")


    #res.to_csv("presentation_2k_31-10.csv")
    if not seaborn:
        res["Topic"] = res["Topic"].astype(str)

        fig = px.scatter(res, x="x", y="y",hover_name="headline",color='Topic',hover_data=["created_at","seo_title","top_n_words","Size"],color_discrete_map={"-1":"black"})
        fig["layout"].pop("updatemenus")
        fig.update_traces(marker=dict(size=12,line=dict(width=.35, color='DarkSlateGrey')),
                          selector=dict(mode='markers'))
        fig.update_layout(height=1200)
        return fig.show()
    else:
        plt.figure(figsize=(40, 15))
        return sns.scatterplot(data=res, x="x", y="y",hue="labels")


interactive(children=(IntSlider(value=3, description='min_cluster_size', max=10, min=1), IntSlider(value=2, de…

In [570]:


def query_embeddings(query_article_id,corpus,raw_df,top_k=10):
    search_results = util.semantic_search(np.array(_pdf.loc[query_article_id].embedding,dtype="double"),
                         np.stack(corpus.embedding.values).astype(np.double),top_k=top_k)
    query_res_df = pd.DataFrame([(_pdf.iloc[res['corpus_id']].name,res['score']) for res in search_results[0]],columns=['article_uid','score']).set_index("article_uid")
    return query_res_df.join(raw_df)[["seo_title",'headline',"created_at","score"]].sort_values(by=['score',"created_at"],ascending=False)

NIZZA =np.array(_pdf.loc["65f65b24d9b4530204775aa8d56eb3a0ad8e1be20e8f756c003684fb512a44ad"].embedding,dtype="double")
search_results = util.semantic_search(NIZZA,
                         np.stack(_pdf.embedding.values).astype(np.double),top_k=12)

query_res_df = pd.DataFrame([(_pdf.iloc[res['corpus_id']].name,res['score']) for res in search_results[0]],columns=['article_uid','score']).set_index("article_uid")
query_res_df.join(df)[["seo_title",'headline',"created_at","score"]].sort_values(by=['score',"created_at"],ascending=False)

,seo_title,headline,created_at,score
article_uid,,,,
65f65b24d9b4530204775aa8d56eb3a0ad8e1be20e8f756c003684fb512a44ad,Messerangriff in Nizza: Ein Toter und mehrere ...,"Ein Toter, mehrere Verletzte bei Messerangriff...",2020-10-29 09:50:15,1.000000
14b86f6e53d53182a55bf6dcc8da0317dc174ead09cbfd50ffe74c4f7b00a8ed,Nizza: Weitere Festnahmen nach Attentat in Kir...,Weitere Festnahmennach Nizza-Anschlag,2020-10-31 16:20:15,0.720966
45d056333acc6b36159222f213a2d5423af81fb9effee87a66bf429a75c8a2f7,Terror in Frankreich: Wie groß ist die ISIS-Ge...,Wie groß ist die ISIS-Gefahr jetzt für Deutsch...,2020-10-29 22:52:16,0.656104
c55c6772d19268c0241fcbe270c5a474999f76888c22ee48c4cdd9aa26bc6756,Familie des Nizza-Attentäters in BILD: Terrori...,Terrorist war erst einen Tag in Frankreich,2020-10-30 23:10:15,0.655760
822e36a5d77a2089df1daac4fbfeedc46adf87067263ef13753bd23b8c5b974a,Rekord-Neuinfektionen: Frankreich plant Lockdo...,Frankreich plant Lockdown für Metropolen,2020-10-26 22:55:15,0.627065
7887b65c403612445245ca86fa4b60fde09af08b062ea3d835d675252cc42d34,Nach Attentat auf Lehrer - Innenminister schli...,Innenminister schließt Pariser Moschee,2020-10-20 04:25:15,0.624687
a8b86da62f6e3b19ef3e203676693dd2280dd7177cc7d84e465e22d9759933c8,Lyon (Frankreich): Schüsse auf Priester an sei...,Schüsse auf Priester an seiner Kirche,2020-10-31 17:27:15,0.618988
ac444425aacfa471522d9d02a4381fcfbf6260627a4b35a148622b1d6e39b89c,Frankreich ruft nach Attacken von Nizza und Av...,Frankreich ruft höchste Terror-Warnstufe aus,2020-10-29 13:06:15,0.616009
0b086a5d828e3b5f1b7f06d5b31f744d2a0946be0889fb12a24b9b6644bb7959,Drei Tote nach Messerangriff: Terror von Nizza...,Weitere Festnahme: Hatte der Attentäter einen ...,2020-10-30 09:49:15,0.611841


In [379]:
query_embeddings("331f9760742a5167b394b320aa089c62d2d537c65be039015899cce417c20bb5",_pdf,df)

,seo_title,headline,created_at,score
article_uid,,,,
331f9760742a5167b394b320aa089c62d2d537c65be039015899cce417c20bb5,Tennis: Nach Gewalt-Vorwürfen seiner Ex – Zver...,Zverev spielt schonwieder Tennis,2020-10-31 22:28:16,1.000000
abcdec3b9326ee7a8aebb1abdf33320e5a827ddf6c628ef482dea377bcbc5c24,Alexander Zverev im BILD-Interview: Unser Tenn...,DIE ZVEREV-ZUKUNFT,2020-10-26 11:09:15,0.735665
7aaaafe9bf67329acd4b0dd48be54b77d9a214d5b60ad2fb619b4340835a1499,VfL Wolfsburg: Josip Brekalo hat Quarantäne üb...,Brekalo hat Quarantäne überstanden,2020-10-29 16:19:16,0.731327
14e98518bed6ed30ba37e6f45d7e87adae8b6cc7ac17a845dd2efc08c5ad2f93,Behrens-Comeback nach Corona-Erkrankung: „Für ...,"„Für Behrens gibt es immer einen Platz!""",2020-10-26 16:16:15,0.730544
b469739b88335782d6f6edf490e5882d1735f651123d9105751688693e3302ac,FCN: Ex-Kapitän Hanno Behrens hat am Freitag d...,Behrens vor Trainings-Comeback,2020-10-22 17:23:15,0.712312
d71f558cb29de2e96e377f679a7b394ceea5868dd5bd1062d3fb3082e1f18fc1,DHfK: Rückraum-Ass Gregor Remke „Wir sind nich...,„Wir sind nicht die Gejagten“,2020-10-21 21:46:15,0.710266
8f15a6792ce6f43abb8029daeaf5f04525b6d7921e46fe5fb6d51e74d779b0b9,"Mesut Özil: Arsenal-Coach Arteta sagt, er sei ...",Arsenal-Trainer: „Bin an Özil gescheitert“,2020-10-22 13:36:15,0.703741
5874d70875a30ddf8ea747cfe6ae4c1d042b51a9e5a3afe52d8a76e39479340b,Undertaker: BILD hat die neue Doku bereits ges...,Was verrät die neue Undertaker-Doku?,2020-10-24 16:31:15,0.699078
b61339c0df9594309c3822494b6ccf35f6f4e20060b9781fb63f5cfe9b891023,Alexander Zverev: Ex-Freundin legt nach: „Ich ...,„Ich lüge nicht – das wissen wir beide!“,2020-10-30 09:25:15,0.698323


In [558]:
os.path.base("./asd/content/arts.csv")


AttributeError: module 'posixpath' has no attribute 'base'

In [336]:
NIZZA =np.array(_pdf.loc["65f65b24d9b4530204775aa8d56eb3a0ad8e1be20e8f756c003684fb512a44ad"].embedding,dtype="double")


['../models/bert-german-dbmdz-uncased-sentence-stsb/umap_100k_6-neighbors_256-comps.pkl',
 '../models/bert-german-dbmdz-uncased-sentence-stsb/umap_100k_12-neighbors_32-comps.pkl',
 '../models/bert-german-dbmdz-uncased-sentence-stsb/umap_100k_12-neighbors_128-comps.pkl',
 '../models/bert-german-dbmdz-uncased-sentence-stsb/umap_100k_6-neighbors_384-comps.pkl',
 '../models/bert-german-dbmdz-uncased-sentence-stsb/umap_100k_19-neighbors_32-comps.pkl',
 '../models/bert-german-dbmdz-uncased-sentence-stsb/umap_100k_6-neighbors_64-comps.pkl',
 '../models/bert-german-dbmdz-uncased-sentence-stsb/umap_100k_12-neighbors_52-comps_0.75-min_dist.pkl',
 '../models/bert-german-dbmdz-uncased-sentence-stsb/umap_100k_6-neighbors_128-comps.pkl',
 '../models/bert-german-dbmdz-uncased-sentence-stsb/umap_100k_6-neighbors_80-comps.pkl']

In [339]:
embeddings.shape

(9839, 768)

In [564]:
_pdf

,0,embedding
article_uid,,
77f34d307c5d8fd10148ab14ff1647e812770638ada4fcf6857bffcbf4bb4012,Tourismus Beauftragter fordert Weihnachten sol...,"[-0.03889315202832222, 0.04216943308711052, 0...."
662b9c5e9a24f491fcdc959f15e1e62018fb47e31209f207ad199ec146320c02,Das Supertalent Dieser Auftritt war zum Fürcht...,"[-0.03217950463294983, -0.28925463557243347, 0..."
a26bac4d411fc5092af65c97024a8f590dc1caf5909b11122ed2e53df1b3d220,Erster Ministerpräsident warnt Kliniken vor de...,"[-0.05921180173754692, -0.1852906346321106, 0...."
0e6b74e1a268960d852e799294c4d707b51d5da424f4b4b61bdfd00943c328e1,Standpunkt Was kommt nach dem Lockdown?. Ab mo...,"[-0.24880725145339966, 0.14853528141975403, 0...."
d304b07fab9d151087ab3a7f08b023f08f0f00eb7ad3c863e2b06dcf930d920d,Tatort Star Barnaby Metschurat sitzt in Frankr...,"[-0.15111424028873444, -0.023308536037802696, ..."
...,...,...
bd7bb4d279f1bb7dcdfe01260708dcd6e230a571615bbc53fb0fe7e0d95edeec,Champions League Atlético Star Carrasco mit Re...,"[-0.20494656264781952, 0.37357446551322937, 0...."
6a621fc818ff0f82a977f44a216ae580b3c3404ddae80cb47e1153bb470c5e0a,Sommerhaus der Stars Dieses Paar gewinnt die R...,"[0.054791416972875595, 0.08964312821626663, -0..."
781affcfcb2d497302d4fd53ba58f9f4e7024092ca4a6ef794a12a2b617919e1,Corona Masken Razzia in Dresdner Straßenbahn. ...,"[-0.23792187869548798, 0.07834027707576752, 0...."


['umap_100k_6-neighbors_256-comps.pkl',
 'umap_100k_12-neighbors_32-comps.pkl',
 'umap_100k_12-neighbors_128-comps.pkl',
 'umap_100k_6-neighbors_384-comps.pkl',
 'umap_100k_19-neighbors_32-comps.pkl',
 'umap_100k_6-neighbors_64-comps.pkl',
 'umap_100k_12-neighbors_52-comps_0.75-min_dist.pkl',
 'umap_100k_6-neighbors_128-comps.pkl',
 'umap_100k_6-neighbors_80-comps.pkl']